In [223]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
import random

In [224]:
data = pd.read_csv('superstore_final.csv',encoding= 'unicode_escape')

In [225]:
data.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,ProductID,Category,Sub-Category,ProductName,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,42433,AG-2011-2040,1/1/2014,6/1/2014,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,...,1000000,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140,35.46,Medium
1,22253,IN-2011-47883,1/1/2014,8/1/2014,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,1000001,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium
2,48883,HU-2011-1220,1/1/2014,5/1/2014,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,...,1000002,Office Supplies,Storage,"Tenex Box, Single Width",66.120,4,0.0,29.640,8.17,High
3,11731,IT-2011-3647632,1/1/2014,5/1/2014,Second Class,EM-14140,Eugene Moren,Home Office,Stockholm,Stockholm,...,1000003,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3,0.5,-26.055,4.82,High
4,22255,IN-2011-47883,1/1/2014,8/1/2014,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,1000004,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.670,5,0.1,37.770,4.70,Medium


In [226]:
data.count()

Row ID            51290
Order ID          51290
Order Date        51290
Ship Date         51290
Ship Mode         51290
Customer ID       51290
Customer Name     51290
Segment           51290
City              51290
State             51290
Country           51290
Postal Code        9994
Market            51290
Region            51290
ProductID         51290
Category          51290
Sub-Category      51290
ProductName       51290
Sales             51290
Quantity          51290
Discount          51290
Profit            51290
Shipping Cost     51290
Order Priority    51290
dtype: int64

In [227]:
print(data['ProductID'].unique().size)
print(data['ProductName'].unique().size)

3788
3788


In [228]:
data['Customer ID'].unique().size

1590

In [229]:
data.isnull().sum()

Row ID                0
Order ID              0
Order Date            0
Ship Date             0
Ship Mode             0
Customer ID           0
Customer Name         0
Segment               0
City                  0
State                 0
Country               0
Postal Code       41296
Market                0
Region                0
ProductID             0
Category              0
Sub-Category          0
ProductName           0
Sales                 0
Quantity              0
Discount              0
Profit                0
Shipping Cost         0
Order Priority        0
dtype: int64

In [230]:
data.columns = [c.replace(' ', '_') for c in data.columns]

In [231]:
data.columns

Index(['Row_ID', 'Order_ID', 'Order_Date', 'Ship_Date', 'Ship_Mode',
       'Customer_ID', 'Customer_Name', 'Segment', 'City', 'State', 'Country',
       'Postal_Code', 'Market', 'Region', 'ProductID', 'Category',
       'Sub-Category', 'ProductName', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping_Cost', 'Order_Priority'],
      dtype='object')

In [232]:
# checking Order_Date  data is one full year because it's better to use a metric per Months or Years in RFM
data['Order_Date'] = pd.to_datetime(data['Order_Date'])
print(data.dtypes)
print(data['Order_Date'].min())
print(data['Order_Date'].max())
print(data.shape)

Row_ID                     int64
Order_ID                  object
Order_Date        datetime64[ns]
Ship_Date                 object
Ship_Mode                 object
Customer_ID               object
Customer_Name             object
Segment                   object
City                      object
State                     object
Country                   object
Postal_Code              float64
Market                    object
Region                    object
ProductID                  int64
Category                  object
Sub-Category              object
ProductName               object
Sales                    float64
Quantity                   int64
Discount                 float64
Profit                   float64
Shipping_Cost            float64
Order_Priority            object
dtype: object
2014-01-01 00:00:00
2014-12-31 00:00:00
(51290, 24)


In [233]:
def review_generate(how_many):
    review = []
    for _ in range(0,how_many):
        review.append(random.randint(0,5))
    return review

In [234]:
review = []
for i in review_generate(51290):
    review.append(i)
data['Review'] = review

## Recency

In [235]:
cust_max_purchase = data.groupby('Customer_ID').Order_Date.max().reset_index()

In [236]:
cust_max_purchase.columns = ['Customer_ID','MaxPurchaseDate']

In [237]:
cust_max_purchase.head()

,Customer_ID,MaxPurchaseDate
0,AA-10315,2014-12-31
1,AA-10375,2014-12-30
2,AA-10480,2014-12-28
3,AA-10645,2014-12-26
4,AA-315,2014-12-29


In [238]:
cust_max_purchase['Recency'] = (cust_max_purchase['MaxPurchaseDate'].max() - cust_max_purchase['MaxPurchaseDate']).dt.days

In [239]:
# get unique cust_id's
data_user = pd.DataFrame(data['Customer_ID'].unique())
data_user.columns = ['Customer_ID']
data_user.count()

Customer_ID    1590
dtype: int64

In [240]:
data_user = pd.merge(data_user, cust_max_purchase[['Customer_ID','Recency']], on='Customer_ID')

In [241]:
data_user.head()

,Customer_ID,Recency
0,TB-11280,13
1,JH-15985,11
2,AT-735,69
3,EM-14140,3
4,PO-18865,0


In [242]:
data_user.Recency.describe()

count    1590.000000
mean       27.738994
std        38.509001
min         0.000000
25%         5.000000
50%        12.000000
75%        34.000000
max       263.000000
Name: Recency, dtype: float64

## Frequency

In [243]:
frequency_df = data.groupby(by=['Customer_ID'], as_index=False)['Order_Date'].count()
frequency_df.columns = ['Customer_ID','Frequency']
frequency_df.head()

,Customer_ID,Frequency
0,AA-10315,42
1,AA-10375,42
2,AA-10480,38
3,AA-10645,73
4,AA-315,8


## Monetary

In [244]:
monetary_df = data.groupby(by='Customer_ID',as_index=False).agg({'Sales': 'sum'})
monetary_df.columns = ['Customer_ID','Monetary']
monetary_df.head()

,Customer_ID,Monetary
0,AA-10315,13747.41300
1,AA-10375,5884.19500
2,AA-10480,17695.58978
3,AA-10645,15343.89070
4,AA-315,2243.25600


In [245]:
#merge recency dataframe with frequency dataframe
temp_df = data_user.merge(frequency_df,on='Customer_ID')
temp_df.head()

,Customer_ID,Recency,Frequency
0,TB-11280,13,15
1,JH-15985,11,68
2,AT-735,69,6
3,EM-14140,3,79
4,PO-18865,0,61


In [246]:

#merge with monetary dataframe to get a table with the 3 columns
rfm_df = temp_df.merge(monetary_df,on='Customer_ID')
#use CustomerID as index
rfm_df.set_index('Customer_ID',inplace=True)
#check the head
rfm_df.head()

,Recency,Frequency,Monetary
Customer_ID,,,
TB-11280,13,15,1574.32200
JH-15985,11,68,18863.40440
AT-735,69,6,403.65000
EM-14140,3,79,21507.29126
PO-18865,0,61,21051.05980


In [247]:
customers_rank = rfm_df
# Create a new column that is the rank of the value of coverage in ascending order
customers_rank['Rank'] = customers_rank['Monetary'].rank(ascending=0)

customers_rank.head()

,Recency,Frequency,Monetary,Rank
Customer_ID,,,,
TB-11280,13,15,1574.32200,1215.0
JH-15985,11,68,18863.40440,116.0
AT-735,69,6,403.65000,1522.0
EM-14140,3,79,21507.29126,59.0
PO-18865,0,61,21051.05980,65.0


In [248]:
customers_rank.sort_values('Rank',ascending=True)

,Recency,Frequency,Monetary,Rank
Customer_ID,,,,
TA-21385,0,65,35668.12080,1.0
GT-14710,1,74,34471.89028,2.0
TC-20980,0,59,34218.26900,3.0
SM-20320,6,39,31125.29496,4.0
BW-11110,19,81,30613.61650,5.0
...,...,...,...,...
MP-7470,114,3,58.11000,1586.0
BD-1500,5,3,52.04700,1587.0
RC-9825,11,1,37.44000,1588.0


## RFM Quartiles

In [249]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,Monetary,Rank
0.25,5.0,12.0,1674.81225,398.25
0.50,12.0,28.0,6248.13559,795.50
0.75,34.0,52.0,13133.09842,1192.75


In [250]:
quantiles.to_dict()

{'Recency': {0.25: 5.0, 0.5: 12.0, 0.75: 34.0},
 'Frequency': {0.25: 12.0, 0.5: 28.0, 0.75: 52.0},
 'Monetary': {0.25: 1674.81225,
  0.5: 6248.135590000002,
  0.75: 13133.098419999998},
 'Rank': {0.25: 398.25, 0.5: 795.5, 0.75: 1192.75}}

In [251]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [252]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

In [253]:
rfm_segmentation.head()

,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile
Customer_ID,,,,,,,
TB-11280,13,15,1574.32200,1215.0,2,2,1
JH-15985,11,68,18863.40440,116.0,3,4,4
AT-735,69,6,403.65000,1522.0,1,1,1
EM-14140,3,79,21507.29126,59.0,4,4,4
PO-18865,0,61,21051.05980,65.0,4,4,4


In [254]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile,RFMScore
Customer_ID,,,,,,,,
TB-11280,13,15,1574.32200,1215.0,2,2,1,221
JH-15985,11,68,18863.40440,116.0,3,4,4,344
AT-735,69,6,403.65000,1522.0,1,1,1,111
EM-14140,3,79,21507.29126,59.0,4,4,4,444
PO-18865,0,61,21051.05980,65.0,4,4,4,444


In [255]:
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

Best Customers:  141
Loyal Customers:  376
Big Spenders:  398
Almost Lost:  56
Lost Customers:  8
Lost Cheap Customers:  167


In [256]:
rfm_segmentation=rfm_segmentation.reset_index()

In [257]:
rfm_segmentation.loc[rfm_segmentation['RFMScore']=='444','Type']= 'Top'
rfm_segmentation.loc[rfm_segmentation['R_Quartile']==1,'Type']= 'Lost'
rfm_segmentation.loc[(rfm_segmentation['R_Quartile']==2) | (rfm_segmentation['R_Quartile']==3) |(rfm_segmentation['F_Quartile']==3) | (rfm_segmentation['F_Quartile']==2) | (rfm_segmentation['M_Quartile']==3) | (rfm_segmentation['M_Quartile']==2) ,'Type']= 'Regular'


In [258]:
## Adding segments to the database
data=pd.merge(data,rfm_segmentation, how ='left', left_on='Customer_ID', right_on='Customer_ID')

In [259]:
top = data[data['Type']=='Top']
lost = data[data['Type']=='Lost']
regular = data[data['Type']=='Regular']

#best_customer_comp.to_csv("C:/Users/yashr/OneDrive/Desktop/best_customer.csv", index=False)

In [260]:
top.to_csv("top.csv", index=False)
lost.to_csv("lost.csv", index=False)
regular.to_csv("regular.csv", index=False)


PermissionError: [Errno 13] Permission denied: 'top.csv'

In [261]:
plot_data = [
    go.Scatter(
        x=rfm_segmentation.query("RFMScore == '111'")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '111'")['Monetary'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=rfm_segmentation.query("RFMScore == '222'")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '222'")['Monetary'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=rfm_segmentation.query("RFMScore == '333' ")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '333'")['Monetary'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'orange',
            opacity= 0.9
           )
    ),
    go.Scatter(
        x=rfm_segmentation.query("RFMScore == '444'")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '444'")['Monetary'],
        mode='markers',
        name='Most',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    )
]

plot_layout = go.Layout(
        yaxis= {'title': "Monetary"},
        xaxis= {'title': "Frequency"},
        title='Frequent customers generate most revenue'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [262]:
plot_data = [
    go.Scatter(
        x=rfm_segmentation.query("RFMScore == '111'")['Recency'],
        y=rfm_segmentation.query("RFMScore == '111'")['Monetary'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=rfm_segmentation.query("RFMScore == '222'")['Recency'],
        y=rfm_segmentation.query("RFMScore == '222'")['Monetary'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=rfm_segmentation.query("RFMScore == '333' ")['Recency'],
        y=rfm_segmentation.query("RFMScore == '333'")['Monetary'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'orange',
            opacity= 0.9
           )
    ),
    go.Scatter(
        x=rfm_segmentation.query("RFMScore == '444'")['Recency'],
        y=rfm_segmentation.query("RFMScore == '444'")['Monetary'],
        mode='markers',
        name='Most',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    )
]

plot_layout = go.Layout(
        yaxis= {'title': "Monetary"},
        xaxis= {'title': "Recency"},
        title='Recent customers generate the most revenue'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [263]:
plot_data = [
    go.Scatter(
        x=rfm_segmentation.query("RFMScore == '111'")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '111'")['Recency'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=rfm_segmentation.query("RFMScore == '222'")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '222'")['Recency'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=rfm_segmentation.query("RFMScore == '333' ")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '333'")['Recency'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'orange',
            opacity= 0.9
           )
    ),
    go.Scatter(
        x=rfm_segmentation.query("RFMScore == '444'")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '444'")['Recency'],
        mode='markers',
        name='Most',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    )
]

plot_layout = go.Layout(
        yaxis= {'title': "Recency"},
        xaxis= {'title': "Frequency"},
        title='Frequent Customers are the Most Recent customers'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

## Customer Lifetime Values

In [264]:
data_group=data.groupby('Customer_ID').agg({'Order_Date': lambda date: (date.max() - date.min()).days,
                                        'Order_ID': lambda num: len(num),
                                        'Quantity': lambda quant: quant.sum(),
                                        'Sales': lambda price: price.sum(),
                                        'Profit':lambda profit: profit.sum()})
data_group.head(5)

,Order_Date,Order_ID,Quantity,Sales,Profit
Customer_ID,,,,,
AA-10315,275,42,145,13747.41300,447.69050
AA-10375,316,42,139,5884.19500,677.47740
AA-10480,351,38,150,17695.58978,1516.47518
AA-10645,348,73,267,15343.89070,3051.43900
AA-315,237,8,20,2243.25600,535.56600


### AOV

In [265]:
data_group['AOV']=data_group['Sales']/data_group['Order_ID']

In [266]:
data_group.head(5)

,Order_Date,Order_ID,Quantity,Sales,Profit,AOV
Customer_ID,,,,,,
AA-10315,275,42,145,13747.41300,447.69050,327.319357
AA-10375,316,42,139,5884.19500,677.47740,140.099881
AA-10480,351,38,150,17695.58978,1516.47518,465.673415
AA-10645,348,73,267,15343.89070,3051.43900,210.190284
AA-315,237,8,20,2243.25600,535.56600,280.407000


### Purchase Frequency

In [267]:
purchase_frequency=sum(data_group['Order_ID'])/data_group.shape[0]

### Repeat Rate and Churn Rate

In [268]:
# Repeat Rate
repeat_rate=data_group[data_group.Order_ID > 1].shape[0]/data_group.shape[0]

In [269]:
#Churn Rate
churn_rate=1-repeat_rate

In [270]:
purchase_frequency,repeat_rate,churn_rate

(32.257861635220124, 0.9955974842767296, 0.004402515723270439)

### Customer Lifetime Value
CLTV = ((Average Order Value x Purchase Frequency)/Churn Rate) x Profit margin.

In [271]:
# Customer Value
data_group['CLV']=(data_group['AOV']*purchase_frequency)/churn_rate

#Customer Lifetime Value
data_group['Cust_Lifetime_Value']=data_group['CLV']*data_group['Profit']

data_group.head()

,Order_Date,Order_ID,Quantity,Sales,Profit,AOV,CLV,Cust_Lifetime_Value
Customer_ID,,,,,,,,
AA-10315,275,42,145,13747.41300,447.69050,327.319357,2.398316e+06,1.073703e+09
AA-10375,316,42,139,5884.19500,677.47740,140.099881,1.026532e+06,6.954521e+08
AA-10480,351,38,150,17695.58978,1516.47518,465.673415,3.412056e+06,5.174298e+09
AA-10645,348,73,267,15343.89070,3051.43900,210.190284,1.540094e+06,4.699504e+09
AA-315,237,8,20,2243.25600,535.56600,280.407000,2.054582e+06,1.100364e+09


In [272]:
data.to_csv("data.csv", index=False)

In [273]:
data.dtypes

Row_ID                     int64
Order_ID                  object
Order_Date        datetime64[ns]
Ship_Date                 object
Ship_Mode                 object
Customer_ID               object
Customer_Name             object
Segment                   object
City                      object
State                     object
Country                   object
Postal_Code              float64
Market                    object
Region                    object
ProductID                  int64
Category                  object
Sub-Category              object
ProductName               object
Sales                    float64
Quantity                   int64
Discount                 float64
Profit                   float64
Shipping_Cost            float64
Order_Priority            object
Review                     int64
Recency                    int64
Frequency                  int64
Monetary                 float64
Rank                     float64
R_Quartile                 int64
F_Quartile